In [1]:
import numpy as np
import pandas as pd
import pyodbc
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
from sklearn import metrics
from sqlalchemy import text


In [2]:
server = "192.168.10.35"
database = "LiveTseDB"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

query = """SELECT [ID]
      ,[Namad]
      ,[StockID]
      ,[NotiTypeName]
      ,[NotiTypeID]
      ,[Time]
      ,[Date]
      ,[LastUpdate]
      ,[IsEvaluate]
      ,[MaxDay0]
      ,[MaxDay1]
      ,[MaxDay2]
      ,[MaxDay3]
      ,[MaxDayPercent0]
      ,[MaxDayPercent1]
      ,[MaxDayPercent2]
      ,[MaxDayPercent3]
      ,[OnEventPrice]
      ,[OnEventPricePercent]
      ,[YeserdayPrice]
      ,[MaxGain]
      ,[MaxGainPosition]
  FROM [LiveTseDB].[dbo].[Notifications] where IsEvaluate = 1;"""

df = pd.read_sql_query(query, conn)
data = pd.read_sql_query(query, conn)

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_19960\1449392340.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
C:\Users\Mesbah\AppData\Local\Temp\ipykernel_19960\1449392340.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [3]:
df.drop(
    [
        "LastUpdate",
        "MaxDay0",
        "MaxDay1",
        "MaxDay2",
        "MaxDay3",
        "ID",
        "IsEvaluate",
        "Namad",
        "NotiTypeName",
        "MaxGainPosition",
        "OnEventPrice",
        "OnEventPricePercent",
        "Time",
        "MaxDayPercent0",
        "MaxDayPercent1",
        "MaxDayPercent2",
        "MaxDayPercent3",
    ],
    inplace=True,
    axis=1,
)

In [4]:
df["Date"] = pd.to_datetime(df["Date"])
df['Date'] = df['Date'].astype('int64') // 10**9 

In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
ohe_data = ohe.fit_transform(df[["NotiTypeID"]])

ohe_df = pd.DataFrame(ohe_data,columns=ohe.get_feature_names_out(["NotiTypeID"]),index=df.index)
    
df = pd.concat([df.drop(columns=["NotiTypeID"]), ohe_df], axis=1)


In [6]:
agg_dict = {col: 'min' if col == "MaxGain" else 'max' 
            for col in df.columns if col not in ['StockID','Date']}

df = pd.concat([df , df.groupby(['StockID', 'Date']).agg(agg_dict).reset_index()] , ignore_index=True)

In [7]:
x = df.drop(["MaxGain"], axis=1)
y = df[["MaxGain"]]

In [8]:
ss = StandardScaler()

x = ss.fit_transform(x)

In [9]:
import joblib

model = joblib.load("model.pkl")

In [10]:
y_test_pred_2 = model.predict(x)

In [ ]:
results = []

new_df = df

new_df["y_pred"] = y_test_pred_2


group_cols = [
    "StockID",
    "NotiTypeID_010E7B33-4D0A-483C-B7A2-53BD6F19A0EC",
    "NotiTypeID_0AD8302F-D2EC-4E30-A26D-FAB20664ECD8",
    "NotiTypeID_45913250-DA33-4A76-A5C4-D8801423A2C4",
    "NotiTypeID_501BC271-1494-44E1-9ED9-65ECAB0768AE",
    "NotiTypeID_5693C959-C56D-4B9B-889D-CD445130182C",
    "NotiTypeID_666D8D60-0F30-46AB-BA13-FF10F271B90C",
    "NotiTypeID_686075F9-C00F-43B3-8F0D-A99F540B2BAB",
    "NotiTypeID_6F14AF33-464D-46B9-83A6-8D779D9EFEDF",
    "NotiTypeID_78F18466-0917-4D2A-AB9F-E2832ADF92A3",
    "NotiTypeID_889313DF-C9E5-45C4-862F-80092885A320",
    "NotiTypeID_96B64AE4-86DE-4CE2-9612-3E4B80CC7B90",
    "NotiTypeID_A68DBD50-3858-476F-B023-67360C2A4BF1",
    "NotiTypeID_CC3EAC84-AA63-4318-8035-AED42A7FF0F9",
    "NotiTypeID_DDC22661-5E87-402D-A487-2CBC46532686",
    "NotiTypeID_EFD74134-5A87-46AC-8A32-9A541497748A",
]


def summarize_group(g):
    return pd.Series(
        {
            "MAE": metrics.mean_absolute_error(
                [g["MaxGain"].mean()], [g["y_pred"].mean()]
            ),
            "MaxGain_min": g["MaxGain"].mean(),
            "y_pred_min": g["y_pred"].mean(),
            "Count": len(g),
        }
    )


group_acc = df.groupby(group_cols).apply(summarize_group).reset_index()
group_acc_sorted = group_acc.sort_values(by="MAE", ascending=False).reset_index(
    drop=True
)

group_acc_sorted.columns = group_acc_sorted.columns.str.replace(r'[^A-Za-z0-9]+', '_', regex=True).str.strip('_')


group_acc_sorted

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_19960\3994637793.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_acc = df.groupby(group_cols).apply(summarize_group).reset_index()


,StockID,NotiTypeID_010E7B33_4D0A_483C_B7A2_53BD6F19A0EC,NotiTypeID_0AD8302F_D2EC_4E30_A26D_FAB20664ECD8,NotiTypeID_45913250_DA33_4A76_A5C4_D8801423A2C4,NotiTypeID_501BC271_1494_44E1_9ED9_65ECAB0768AE,NotiTypeID_5693C959_C56D_4B9B_889D_CD445130182C,NotiTypeID_666D8D60_0F30_46AB_BA13_FF10F271B90C,NotiTypeID_686075F9_C00F_43B3_8F0D_A99F540B2BAB,NotiTypeID_6F14AF33_464D_46B9_83A6_8D779D9EFEDF,NotiTypeID_78F18466_0917_4D2A_AB9F_E2832ADF92A3,NotiTypeID_889313DF_C9E5_45C4_862F_80092885A320,NotiTypeID_96B64AE4_86DE_4CE2_9612_3E4B80CC7B90,NotiTypeID_A68DBD50_3858_476F_B023_67360C2A4BF1,NotiTypeID_CC3EAC84_AA63_4318_8035_AED42A7FF0F9,NotiTypeID_DDC22661_5E87_402D_A487_2CBC46532686,NotiTypeID_EFD74134_5A87_46AC_8A32_9A541497748A,MAE,MaxGain_min,y_pred_min,Count
0,37614886280396031,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,28.758198,32.989691,4.231492,1.0
1,37614886280396031,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,13.745659,18.556369,4.810710,2.0
2,16553062355259729,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,10.301082,0.646831,10.947912,1.0
3,64216772923447100,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,10.244511,12.362637,2.118126,1.0
4,17914401175772326,1,0,0,0,1,1,1,1,0,0,1,0,1,0,0,9.980200,9.921799,-0.058402,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10166,22312990497291517,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.001182,4.482589,4.481407,12.0
10167,39436183727126211,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000866,5.079295,5.080161,3.0
10168,59522975635235187,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.000505,5.556652,5.556148,42.0
10169,19298748452450329,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0.000191,7.046477,7.046286,1.0


In [36]:
from sqlalchemy import create_engine
import urllib.parse

database = "Boors"
server = "njweb"
username = "Boors"
password = urllib.parse.quote_plus("Boors2025@")
driver = "ODBC+Driver+18+for+SQL+Server"

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}"
    "&Encrypt=no"
    "&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

with engine.connect() as conn:
    group_acc_sorted.to_sql("boorsModelInputs", con=conn, if_exists="replace", index=False)